## Save selected movie from lif in zarr format.

In [3]:
from readlif.reader import LifFile
from numcodecs import Blosc
import numpy as np
from dask import delayed
import dask.array as da

import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

In [4]:
# specify the path to the .lif file
exp_path = r'R:\Dante\hillman_academy_2023\Jinny_test.lif'
# unique identifier for the experiment
# if unknown, print names and dimensions of objects in lif to find the correct one
sel_name = 'TileScan 5/C4_Merged'

# specify compressor for zarr
compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE)

# output path for zarr
zarr_dir = r'R:\Kasia\tracking\TrackGardener'
zarr_name = 'C4'
channel_list = [0,1]
# specify which resolution level to start with, 0 - original data
res_level = 1

# specify zarr chunking 
size_t = 10
size_xy = 512

In [5]:
# load experiment
exp = LifFile(exp_path)

In [6]:
# get names and dimensions of all elements in the lif file
[[image.name, image.dims] for image in exp.get_iter_image()]

[['G6 Region1', Dims(x=2048, y=2048, z=1, t=1, m=76)],
 ['G6 Region1_Merged', Dims(x=35181, y=7548, z=1, t=1, m=1)],
 ['TileScan 4/B4 Region1', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/C4', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/D6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/E6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/F6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/G6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/B4 Region1_Merged', Dims(x=16801, y=16798, z=1, t=1, m=1)],
 ['TileScan 4/C4_Merged', Dims(x=16791, y=16793, z=1, t=1, m=1)],
 ['TileScan 4/D6_Merged', Dims(x=16799, y=16800, z=1, t=1, m=1)],
 ['TileScan 4/E6_Merged', Dims(x=16809, y=16796, z=1, t=1, m=1)],
 ['TileScan 4/F6_Merged', Dims(x=16800, y=16794, z=1, t=1, m=1)],
 ['TileScan 4/G6_Merged', Dims(x=16792, y=16783, z=1, t=1, m=1)],
 ['TileScan 1/B4 Region1', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 1/C4', Dims(x=2048, y=2048, z=1, t=1, m=77)],

In [7]:
# get a a selected series
t = [image for image in exp.get_iter_image() if sel_name in image.name][0]
t

'LifImage object with dimensions: Dims(x=16796, y=16798, z=1, t=241, m=1)'

In [8]:
# function for lazy loading of frames
def frame_getter(lif,ind,channel=0):

    return np.array(lif.get_frame(z=0, t=ind, c=channel))

In [9]:
# run to check if the stack is read correctly

lazy_arrays = [delayed(frame_getter)(t,i) for i in range(t.dims.t)]
dask_arrays = [da.from_delayed(delayed_reader, shape=[t.dims.y,t.dims.x], dtype='uint16') for delayed_reader in lazy_arrays]
stack = da.stack(dask_arrays, axis=0)
stack

dask.array<stack, shape=(241, 16798, 16796), dtype=uint16, chunksize=(1, 16798, 16796), chunktype=numpy.ndarray>

In [ ]:
# iterate through channels and save to zarr
for channel in channel_list:

    lazy_arrays = [delayed(frame_getter)(t,i,channel) for i in range(t.dims.t)]
    dask_arrays = [da.from_delayed(delayed_reader, shape=[t.dims.y,t.dims.x], dtype='uint16') for delayed_reader in lazy_arrays]
    stack = da.stack(dask_arrays, axis=0)
    
    # subsample if requested
    if res_level > 0:
        sub_sample = 2**res_level
        stack = stack[:,::sub_sample,::sub_sample]

    # write the image data
    zarr_path = f"{zarr_dir}/{zarr_name}_ch{str(channel).zfill(2)}.zarr"
    store = parse_url(zarr_path, mode="w").store
    root = zarr.group(store=store)

    # it will fail if the store already contains arrays
    write_image(image=stack, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy),compressor=compressor))